In [3]:
#libraries
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense , Dropout
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import matplotlib.pyplot as plt
from glob import glob  



In [10]:
train_path = "../input/fruits/fruits-360_dataset/fruits-360/Training"
test_path = "../input/fruits/fruits-360_dataset/fruits-360/Test"



In [11]:
img = load_img(train_path + "/Apple Braeburn/0_100.jpg")
plt.imshow(img)
plt.axis("off")# eksen isimleri gizlendi.
plt.show()



In [12]:
x = img_to_array(img)
print(x.shape)


In [13]:

className = glob(train_path + '/*') # train dosyasının altında bulunan (/*) her dosyanın ismini className'in içine aktar. 
numberOfClass = len(className)
print("NumberOfClass : ", numberOfClass)

In [15]:
# Dropout
# CNN MODEL
model = Sequential()
model.add(Conv2D(32, (3,3), input_shape = x.shape))
model.add(Activation("relu"))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D())

model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D())

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(numberOfClass)) # output layer
model.add(Activation("softmax"))

In [16]:
model.compile(loss = "categorical_crossentropy", # loss fonksiyonu softmax(mullticlass) kullanıldığı için categorical_crossentropy oldu
              optimizer = "rmsprop",
              metrics = ["accuracy"])

In [17]:
batch_size = 32 # 32 fotoğrafta bir aynı işlemler tekrarlanıyor.

In [18]:
# DATA GENERATION - train - test
train_data_generate = ImageDataGenerator(rescale = 1./255 ,
                          shear_range = 0.3 , # hafif bir açı ile döndürme
                          horizontal_flip= True,
                          zoom_range = 0.3)
test_data_generate = ImageDataGenerator(rescale = 1./255)

train_generator = train_data_generate.flow_from_directory(
    train_path, 
    target_size = x.shape[:2], # x.shape : (100,100,3) rgb'nin alınmasına gerek yok
    batch_size = batch_size,
    color_mode = "rgb",
    class_mode = "categorical")

test_generator = test_data_generate.flow_from_directory(
    test_path, 
    target_size = x.shape[:2], # x.shape : (100,100,3) rgb'nin alınmasına gerek yok
    batch_size = batch_size,
    color_mode = "rgb",
    class_mode = "categorical")

In [19]:
#fit
hist = model.fit_generator(
        generator = train_generator,
        steps_per_epoch = 1600 // batch_size,
        epochs = 100, 
        validation_data = test_generator,
        validation_steps = 800 // batch_size)

In [21]:
# model save
model.save_weights("cnn_fruit_hist.h5")

In [22]:
# model evaluation
print(hist.history.keys())
plt.plot(hist.history["loss"], label = "Train Loss")
plt.plot(hist.history["val_loss"], label = "Validation Loss")
plt.legend()
plt.show()
plt.figure()
plt.plot(hist.history["accuracy"], label = "Train accuracy")
plt.plot(hist.history["val_accuracy"], label = "Validation accuracy")
plt.legend()
plt.show()

In [24]:
 # save history
import json
with open("cnn_fruit_hist.json" , "w") as f:
    json.dump(hist.history , f)

In [26]:
# load history 
import codecs
with codecs.open("./cnn_fruit_hist.json" , "r" , encoding="utf-8") as f:
    h = json.loads(f.read())
plt.plot(h["loss"], label = "Train Loss")
plt.plot(h["val_loss"], label = "Validation Loss")
plt.legend()
plt.show()
plt.figure()
plt.plot(h["accuracy"], label = "Train accuracy")
plt.plot( h["val_accuracy"], label = "Validation accuracy")
plt.legend()
plt.show()
    
    